In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.optimize import nnls
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer

df = pd.read_csv('./Non-square-sp-smdm-memory.csv',encoding='euc-kr')
df

,lr,lc,rc,ld,rd,lnnz,rnnz,저장전메모리(작성),저장후메모리(작성),"좌측, 우측 희소행렬 저장",todense후메모리(작성),우측 희소행렬 toDense,곱셈후메모리(작성),곱셈,총메모리합
0,11636,20031,11259,0.016000,0.022117,3729601,4988188,10.70,15.6,4.90,15.7,0.1,15.8,0.1,5.10
1,472,155935,3626,0.042674,0.005037,3141609,2848497,9.98,12.4,2.42,15.2,2.8,15.3,0.1,5.32
2,34369,19457,9425,0.002993,0.017991,2001484,3299434,10.80,14.3,3.50,14.4,0.1,16.2,1.8,5.40
3,17922,160686,714,0.001152,0.038738,3317532,4447011,10.30,15.0,4.70,15.0,0.0,16.0,1.0,5.70
4,9581,16824,19275,0.047721,0.008130,7693122,2636477,10.50,17.7,7.20,17.8,0.1,17.9,0.1,7.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5579,138770,12555,0.020277,0.030960,15700344,53942019,10.10,56.3,46.20,69.3,13.0,69.4,0.1,59.30
96,26061,51521,37087,0.037411,0.026095,50231125,49861395,10.50,77.6,67.10,77.6,0.0,77.6,0.0,67.10
97,35625,79414,11287,0.016000,0.054465,45267635,48824425,10.80,79.1,68.30,79.1,0.0,79.2,0.1,68.40
98,25865,65104,18847,0.035231,0.045996,59328892,56438830,10.60,77.3,66.70,86.5,9.2,90.2,3.7,79.60


In [2]:
# feature
feature = df[['lr','lc','rc','ld','rd','lnnz','rnnz']] 
target = df['총메모리합']

In [7]:
# 훈련 데이터와 테스트 데이터 9:1 분리
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size = 0.1)

In [8]:

# 탐색 대상 함수 (XGBRegressor)
def XGB(
max_depth,
learning_rate, 
n_estimators, 
min_child_weight, 
subsample,
_lambda,
#gamma ,
#colsample_bytree, 
#_alpha,
silent=True, 
n_jobs=-1):
    
    # 모델 정의
    model = xgb.XGBRegressor( 
objective = 'reg:squarederror',
max_depth=int(max_depth),
learning_rate=learning_rate,
n_estimators=int(n_estimators),
min_child_weight=min_child_weight,
subsample=subsample,
reg_lambda=_lambda,        
#gamma=gamma,
#colsample_bytree=colsample_bytree, 
#reg_alpha=_alpha
n_jobs=n_jobs        
                              )
    
    # bayesian optimization을 통해 파라미터를 받아
    # Train을 Train + Validation으로 나눠 cross-validation 성능 확인
    kfold = KFold(n_splits=5, shuffle = True, random_state=0)

    # cross-validation 평균 성능 성능 확인
    score = cross_val_score(model,
                            X_train, 
                            y_train, 
                            cv=kfold,
                            scoring=make_scorer(custom_scoring,greater_is_better=False),
                            n_jobs=-1
                           ).mean()
    
    return score

In [9]:
from sklearn.metrics import r2_score, mean_squared_error

# Metric
def mean_absolute_percentage_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

def custom_scoring(real, pred):
    rmse = np.sqrt(mean_squared_error(real, pred))
    r2 = r2_score(real, pred)
    mape = mean_absolute_percentage_error(real, pred)
    return mape

In [10]:
from bayes_opt import BayesianOptimization

# # 하이퍼파라미터 정의
# pbounds = {
# 'max_depth': (11, 14), 
# 'learning_rate': (0.01, 0.3),
# 'n_estimators': (50, 100),
# 'min_child_weight': (0.1, 1),
# 'subsample': (0.5, 1), 
# '_lambda' : (0.1,1),    
# #'gamma': (0, 0.3),     
# #'colsample_bytree' :(0.5, 1)
# #'_alpha' : (0,1)           
#                       }
# 하이퍼파라미터 정의
pbounds = {
'max_depth': (12, 16), 
'learning_rate': (0.01, 0.3),
'n_estimators': (50, 150),
'min_child_weight': (0.1, 1),
'subsample': (0.5, 1), 
'_lambda' : (0.1,1),    
#'gamma': (0, 0.3),     
#'colsample_bytree' :(0.5, 1)
#'_alpha' : (0,1)           
                      }

# Bayesian optimization 객체 생성
bo=BayesianOptimization(f=XGB, pbounds=pbounds, verbose=2, random_state=1 )    

# 메소드를 이용해 최대화 과정 수행 (파라미터 넣고 목적함수 값 출력하고)
bo.maximize(init_points=2, n_iter=50, acq='ei', xi=0.01)

# 뽑힌 최적의 하이퍼파라미터 값 확인
print("{}\n".format(bo.max))

|   iter    |  target   |  _lambda  | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
|  1        | -15.82    |  0.4753   |  0.2189   |  12.0     |  0.3721   |  64.68    |  0.5462   |
|  2        | -14.63    |  0.2676   |  0.1102   |  13.59    |  0.5849   |  91.92    |  0.8426   |
|  3        | -14.75    |  0.2233   |  0.1116   |  13.92    |  0.4265   |  91.26    |  0.7918   |
|  4        | -15.1     |  0.616    |  0.2067   |  12.15    |  0.9256   |  94.65    |  0.8862   |
|  5        | -37.7     |  0.1      |  0.01     |  12.02    |  1.0      |  91.5     |  1.0      |
|  6        | -14.19    |  0.3258   |  0.1503   |  14.26    |  0.3906   |  91.96    |  0.7704   |
|  7        | -16.18    |  0.5268   |  0.2203   |  13.87    |  0.4454   |  93.26    |  0.7482   |
|  8        | -17.58    |  1.0      |  0.3      |  14.18    |  1.0      |  91.8     |  0.5      |
|  9        | -15.0 

|  51       | -15.75    |  0.1816   |  0.1234   |  13.92    |  0.1866   |  96.93    |  0.7351   |
|  52       | -14.36    |  0.1305   |  0.09845  |  12.22    |  0.6377   |  128.4    |  0.633    |
{'target': -14.074444609298627, 'params': {'_lambda': 0.32902847452242395, 'learning_rate': 0.05788539762274639, 'max_depth': 13.612870162900137, 'min_child_weight': 0.18868310294529117, 'n_estimators': 96.79388709797425, 'subsample': 0.7164282712398647}}



In [11]:
import xgboost as xgb

# Train + Valid cross-validation을 거친, 최적의 하이퍼파라미터를 사용
best_model = xgb.XGBRegressor(
objective = 'reg:squarederror',
max_depth=int(bo.max['params']['max_depth']),
learning_rate=bo.max['params']['learning_rate'],
n_estimators=int(bo.max['params']['n_estimators']),
min_child_weight=bo.max['params']['min_child_weight'],
subsample=bo.max['params']['subsample'],
reg_lambda = bo.max['params']['_lambda'],    
#gamma=bo.max['params']['gamma'],
#colsample_bytree=bo.max['params']['colsample_bytree'],
#reg_alpha = bo.max['params']['_alpha']
n_jobs=-1
                             )
# 모델 훈련
best_model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05788539762274639, max_delta_step=0, max_depth=13,
             min_child_weight=0.18868310294529117, missing=nan,
             monotone_constraints='()', n_estimators=96, n_jobs=-1,
             num_parallel_tree=1, random_state=0, reg_alpha=0,
             reg_lambda=0.32902847452242395, scale_pos_weight=1,
             subsample=0.7164282712398647, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [12]:
# 최적의 하이퍼파라미터 값 확인
print("{}\n".format(bo.max))

# 훈련데이터 예측
y_train_pred = best_model.predict(X_train)
print("-------- 훈련데이터 예측 --------------------------")
print("rmse : {}".format(np.sqrt(mean_squared_error(y_train, y_train_pred))))
print("mape : {}%".format(mean_absolute_percentage_error(y_train, y_train_pred)))
print("\n")

# 검증데이터 예측
print("-------- 검증데이터 예측 --------------------------")
print("mape : {}%".format(-bo.max['target']))
print("\n")

# 테스트데이터 예측
y_pred = best_model.predict(X_test)
print("-------- 테스트데이터 예측 -------------------------")
print("rmse : {}".format(np.sqrt(mean_squared_error(y_test, y_pred))))
print("mape : {}%".format(mean_absolute_percentage_error(y_test, y_pred)))
print("\n")

{'target': -14.074444609298627, 'params': {'_lambda': 0.32902847452242395, 'learning_rate': 0.05788539762274639, 'max_depth': 13.612870162900137, 'min_child_weight': 0.18868310294529117, 'n_estimators': 96.79388709797425, 'subsample': 0.7164282712398647}}

-------- 훈련데이터 예측 --------------------------
rmse : 0.4462145597382869
mape : 0.7766231201214279%


-------- 검증데이터 예측 --------------------------
mape : 14.074444609298627%


-------- 테스트데이터 예측 -------------------------
rmse : 4.974654275823156
mape : 11.274054423148476%


